In [1]:
import torch
from pyro.distributions import constraints
from pyroapi import pyro

import pyro.distributions as dist
from tqdm import trange

pyro.clear_param_store()

In [2]:
raw_data = {
    "Student athlete": {"Should be paid": 80, "Should not be paid": 20},
    "Student nonathlete": {"Should be paid": 220, "Should not be paid": 80}
}
data = torch.concat((torch.tensor([True, True]).repeat(80, 1), torch.tensor([True, False]).repeat(20,1), torch.tensor([False, True]).repeat(220,1), torch.tensor([False, False]).repeat(80,1)))
data, data.size()

(tensor([[ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],
         [ True,  True],


In [3]:
def model_paying_athletes():
    student = pyro.sample("student", dist.Categorical(logits=torch.zeros(400)))
    p = pyro.sample("latent_paying", dist.Beta(concentration0=100., concentration1=300.))
    pyro.sample("obs", dist.Bernoulli(p), obs=data[student][0].clone().detach().float())

In [4]:
def guide_paying_athletes():
    pyro.sample("student", dist.Categorical(logits=torch.zeros(400)))
    alpha = pyro.param("alpha", torch.tensor(100.), constraint=constraints.positive)
    beta = pyro.param("beta", torch.tensor(300.), constraint=constraints.positive)
    pyro.sample("latent_paying", dist.Beta(concentration0=alpha, concentration1=beta))

In [5]:
optim = pyro.optim.Adam({"lr": 0.025})
loss = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(model_paying_athletes, guide_paying_athletes, optim, loss)

In [ ]:
losses = []
params = []

for _ in trange(10_000):
    l = svi.step()
    losses.append({"loss": l})
    param = {"alpha": pyro.param("alpha").item(), "beta": pyro.param("beta").item()}
    params.append(param)

  4%|▎         | 367/10000 [00:01<00:54, 178.28it/s]

In [ ]:
pyro.param("alpha"), pyro.param("beta")

In [ ]:
import pandas as pd
import matplotlib as plt

In [ ]:
df = pd.DataFrame(losses)

df.plot()

In [ ]:
df = pd.DataFrame(params)

df.plot()